## Summarize NDFD Element by HexBin
This notebook summarizes National Weather Service's National Digitial Forecast Database (NDFD) datasets polygon shapefile that consists of hexagons that cover all of California.
The 'input_dicts' list is a list of dictionaries that provide information on the NDFD GRIB files to download and process, the type of summary stastic that will be generated for each hexagon, and the item IDs for each output ArcGIS Online Feature Service that will be updated with the summary information. Addtional details on the input dictonaries is provided below.  

To run this notebook you must:
1. Run the notebook within python environment with both the ArcGIS for Python API and the ArcPy libaries installed (we are currently using ann ArcGIS Online Advanced Python Notebook)
2. Create the output feature services by publishing out the hexagon shapefile for each NDFD element/variable and time period.  These feature service will need to include the 'grid_id' field from the hexagon shapefile along with a 'Value' field with a double data type.  
3. Update the input parameters like the download folder, Zonal Tables FGDB path, and the input hexbin path.
4. Update the input dictonaries to match the NDFD variables that you want to summaries, and the item ids to feature services you want to update.




In [ ]:
# Import Libaries
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta
import arcpy
from arcpy.sa import *
from arcgis.features import GeoAccessor, GeoSeriesAccessor, FeatureSet
from arcgis.gis import GIS
import requests
#from tqdm.notebook import tqdm
import pytz
import time
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline

# Set Start Date/Time to Track how long the notebook takes to run
from datetime import datetime
start_time = datetime.now()

### Update the Input Parameters

In [ ]:
# Input Parameters
# Login with current GIS User
gis = GIS("home")

# Allow ArcPy to Overwrite Outputs?
arcpy.env.overwriteOutput = True

# Download Folder - Where should the GRIB Files be Stored?
download_folder = r"C:\git\ndfd_processing\NDFD"

# What it might be on AGOL
# download_folder = r"/arcgis/home/weather_data/NDFD/"

# Set Up FGDB to Store Zonal Stats Tables
# Check to see if FGDB Exists to Store Zonal Stats Tables, if not create it
if arcpy.Exists(r"C:\git\ndfd_processing\TEMP\ZonalTables.gdb"):
    print("Zonal Table Aready Exists, Skipping")
else:
    arcpy.management.CreateFileGDB(r"C:\git\ndfd_processing\TEMP", "ZonalTables.gdb")

# What it might be on AGOL
# Check to see if FGDB Exists to Store Zonal Stats Tables, if not create it
#if arcpy.Exists("/arcgis/home/weather_data/ZonalTables.gdb"):
#    print("Zonal Table Aready Exists, Skipping")
#else:
#    arcpy.management.CreateFileGDB("/arcgis/home/weather_data/", "ZonalTables.gdb")


# Input Hexagons - What HexBin Data Should we Use?    
input_hexbin = r"C:\git\ndfd_processing\HEXBIN\hexbin.shp"

# What it might be on AGOL 
# What HexBin Data Should we Use?    
#input_hexbin = "/arcgis/home/weather_data/hexbin.shp"

### Update Input Dictonaries
Each dictonary in the 'input_dicts' list represents a new NDFD Data Element/Variable to Process, please take a look at the list below to see some examples. 
* *Name*: The name you want to associate with the NDFD Data Element/Variable you are Summarizing
* *URL*: The URL to the downloadable GRIB file
* *Variable*: The variable name that you want to summarize. Hint: There is typically only one variable included in the NDFD GRIB Datasets, just open the *.bin file in ArcGIS Pro and use any of hte multidimenisal raster tools to find its name.
* *Output Layer*: The name of the temporary output raster layer that is used in the analysis
* *Output Table*: The path and name of the zonal statics as table tool output that is used in the analysis.
* *Item IDs*: A dictonary that lists all the days/time periods you want to summarize and the corrosponding item ids for the feature service that will be updated with this information.
* *Summary Type*: The type of summary statistic that you want to use to aggergate the NDFD GRIB data to the hexagons (MIN, MAX, etc.)
* *Conversion*: Optional element if you wish to apply a conversion to the data.  Current options include 'Kelvin to Fahrenheit' and 'Meters per Second to Miles per Hour'.
* *Rounding*: Optional element if you wish to round the summary values.


In [ ]:
# Input Parameters
inputs_dicts = [{'Name':'Maximum Daily Temperature (NDFD)', 
                'URL':'https://tgftp.nws.noaa.gov/SL.us008001/ST.opnl/DF.gr2/DC.ndfd/AR.conus/VP.001-003/ds.temp.bin', 
                'Variable':'T@HTGL', 
                'Output Layer':'temp_layer', 
                'Output Table':'/arcgis/home/weather_data/ZonalTables.gdb/MaxTempTable', 
                'Item IDs': {'Today': '8714f080ce6c4dc39d2819ded9e519d1', 
                             'Tomorrow': 'f435e9a75e22471d978ec0fd80cd7be3', 
                             'Day After Tomorrow (Ends at 5pm)': '2d6d87c6dd694b9994d53264dcfdc15e', 
                             'Entire 3 Day Period': '503c812174534c1ca6ce2ad73809f269'}, 
                'Summary Type':'MAX', 
                'Conversion':'Kelvin to Fahrenheit', 
                 'Rounding':1},
               {'Name':'Minimum Daily Relative Humidity (NDFD)', 
                'URL':'https://tgftp.nws.noaa.gov/SL.us008001/ST.opnl/DF.gr2/DC.ndfd/AR.conus/VP.001-003/ds.rhm.bin', 
                'Variable':'RH@HTGL', 
                'Output Layer':'minrh_layer', 
                'Output Table':'/arcgis/home/weather_data/ZonalTables.gdb/minrh_zonal_stats', 
                'Item IDs': {'Today': '4becbf9861314407b23a94a15f51e6e4', 
                             'Tomorrow': '09812c61da4d408386cdb814d4a68767', 
                             'Day After Tomorrow (Ends at 5pm)': '55ecdcc0f90d4bda966ccf6e1b66c5da',
                             'Entire 3 Day Period': 'a8378aaae8bb4070a9192a2449c59ba2'}, 
                'Summary Type':'MIN', 
                'Rounding':1},
              {'Name':'Maximum Daily Wind Speed (NDFD)', 
                'URL':'https://tgftp.nws.noaa.gov/SL.us008001/ST.opnl/DF.gr2/DC.ndfd/AR.conus/VP.001-003/ds.wspd.bin', 
                'Variable':'WindSpd@HTGL', 
                'Output Layer':'windspeed_layer', 
                'Output Table':'/arcgis/home/weather_data/ZonalTables.gdb/windspeed_zonal_stats', 
                'Item IDs': {'Today': '769c8c14fc0947a6aa4e2ca7dfceb2ae', 
                             'Tomorrow': 'f92e185e15024979a386e554bdcb6e69', 
                             'Day After Tomorrow (Ends at 5pm)': '5dc72303989e4b1bb918e7b3e37a48fc', 
                             'Entire 3 Day Period': '1444cceb9fe04842922a30301b33256c'}, 
                'Summary Type':'MAX', 
               'Conversion':'Meters per Second to Miles per Hour', 
               'Rounding':1}]

### Run Process to Read and Summarize the NDFD Datasets by Hexagon

In [ ]:
# Create a Spatially-Enabled DataFrame of the Hexbins
hex_sdf = pd.DataFrame.spatial.from_featureclass(input_hexbin)
hex_sdf

In [ ]:
# Functions

# Decorator to Check Run Time
def check_runtime(func):
    """Function to check the runtime of other functions."""
    def wrapper(*args, **kwargs):
        func_start = datetime.now()
        result = func(*args, **kwargs)
        func_end = datetime.now()
        print(f"Function {func.__name__} Took {round((func_end - func_start).seconds, 2)} Seconds to Run")
        return result
    return wrapper


def kelvin2Fahrenheit(k):
    """Function to convert tempature from Kelvin to Fahrenheit"""
    f = ((k-273.15)*(float(9)/float(5)))+32
    return f

def meters2mph(x):
    """Function to conver from Meters per Second and Miles per Hour"""
    y = float(x)*2.23694
    return y

# Download File
def downloadGRIB(input_url, download_folder):
    """Download the GRIB File
        Parameters:
            input_url (str): The URL of the GRIB File to Download
            download_folder (str): The path to the folders that will contain the downloaded GRIB files
        Returns:
            str: The path to the downloaded GRIB file.
    """
    output_file = os.path.basename(input_url)
    grib_file = os.path.join(download_folder, output_file)
    r = requests.get(input_url, allow_redirects=True)
    open(grib_file, 'wb').write(r.content)
    return grib_file

# Create Raster Layer from GRIB
def makeRasterLayer(input_file, output_layer, variable):
    """
    Create a Raster Layer from a Multidimensional Raster File
    Parameters:
        input_file (str): The path to the GRIB or Other Multidimensional Raster File
        output_layer (str): The name of the raster layer that will be created
        variable (str): The variable that will be used to create the raster layer
    Returns:
        str: The name of the raster layer that was created
    """
    arcpy.md.MakeMultidimensionalRasterLayer(input_file, output_layer, variable, "ALL")
    return output_layer

# Run Zonal Stats
def runZonalStats(input_hexbin, output_layer, output_table, id_field="GRID_ID"):
    """
    Run Zonal Stastistics As Table using the hexbin polygons and the raster layer
    Parameters:
        input_hexbin (str): input hexbins???
    """
    outZSaT = ZonalStatisticsAsTable(input_hexbin, id_field, output_layer, output_table, "NODATA", "ALL", "ALL_SLICES")
    return output_table

# Categorize Timestamp
def categorizeStdTime(t):
    """Calculates the difference (in days)  between the analysis date adn the current date. Assigns the date a category."""
    #date_dif = (t.date() - datetime.now().date()).days
    date_dif = (t.date() - datetime.now(pytz.timezone('America/Los_Angeles')).date()).days
    if date_dif == 0:
        category = 'Today'
    elif date_dif == 1:
        category = 'Tomorrow'
    elif date_dif == 2:
        category = 'Day After Tomorrow (Ends at 5pm)'
    else:
        category = 'Other/Unknown'
    return category


def summarizeTimePeriods(df, summary_type, all_periods_name, groupby_column = 'Time Period', time_periods=['Today', 'Tomorrow', 'Day After Tomorrow (Ends at 5pm)']):
    output_dict = {}
    summary_stat = summary_type.lower()
    #time_periods = list(df[groupby_column].unique())
    df = df[df['Time Period'].isin(time_periods)]
    for t in time_periods:
        temp_df = df[df['Time Period'] == t]
        summary_df = temp_df.groupby('grid_id').agg({summary_type:summary_stat})
        summary_df.reset_index(inplace=True)
        # summary_hexbin = hexbin_sdf.merge(summary_df, on='grid_id', how='left')
        # Clean Up Foramting
        #rename_dict = {'grid_id':'GRID_ID', 'SHAPE':'SHAPE', summary_type:'Value'}
        rename_dict = {'grid_id':'GRID_ID', summary_type:'New Value'}
        summary_df.rename(columns=rename_dict, inplace=True)
        summary_df = summary_df[list(rename_dict.values())]
        # Add to Dictonary
        output_dict[t] = summary_df
        # Clean Up
        del temp_df, summary_df #, summary_hexbin
    # Search Across the Entire Time Period if all_periods_name is not none
    if all_periods_name is not None:
        summary_df = df.groupby('grid_id').agg({summary_type:summary_stat})
        summary_df.reset_index(inplace=True)
        #summary_hexbin = hexbin_sdf.merge(summary_df, on='grid_id', how='left')
        rename_dict = {'grid_id':'GRID_ID', 'SHAPE':'SHAPE', summary_type:'Value'}
        rename_dict = {'grid_id':'GRID_ID', summary_type:'New Value'}
        summary_df.rename(columns=rename_dict, inplace=True)
        summary_df = summary_df[list(rename_dict.values())]
        output_dict[all_periods_name] = summary_df
        del summary_df#, summary_hexbin
    # Clean Up Formatting
    return output_dict



def updateFeatureService(gis, sdf, agol_item_id, layer_num=0, chunkSize=100):
    item = gis.content.get(agol_item_id)
    layer = item.layers[layer_num]
    
    current_sdf = pd.DataFrame.spatial.from_layer(layer)
    current_columns = current_sdf.columns.to_list()
    # Columns to remove
    removal_list = ['OBJECTID', 'Shape__Area', 'Shape__Length', 'CreationDate', 'Creator', 'EditDate', 'Editor']
    current_columns = list(filter(lambda i: i not in removal_list, current_columns))
    sdf = sdf[current_columns]
    
    print(f"Checking Inputs to Compare SDF, Number of Records {sdf.shape[0]}")
    print(sdf.head())
    
    difference_sdf = compareLayer2SDF(layer, sdf, id_field = "GRID_ID", value_field="Value")
    
    print(f"Difference SDF, Number of Records {difference_sdf.shape[0]}")
    print(difference_sdf.head())
    
    updateResults = []
    if difference_sdf is not None and len(difference_sdf)>0:
        print(f"Update Features {len(difference_sdf)}")
        numAddLoops = math.floor(len(difference_sdf)/chunkSize) + 1
        startAddNum = 0
        while numAddLoops > 0:
            minAddNum = startAddNum
            maxAddNum = startAddNum + chunkSize
            startAddNum = startAddNum + chunkSize
            numAddLoops = numAddLoops-1
            temp_fs = difference_sdf[minAddNum:maxAddNum].copy()
            update_featureset = FeatureSet.from_dataframe(temp_fs)
            updateFeatures = layer.edit_features(updates=update_featureset)
            updateResults.append(updateFeatures)
    else:
        print("No Records to Add")
    return updateResults

def truncateLoadFeatureService(gis, sdf, agol_item_id, layer_num=0, chunkSize=100):
    item = gis.content.get(agol_item_id)
    layer = item.layers[layer_num]
    
    current_sdf = pd.DataFrame.spatial.from_layer(layer)
    current_columns = current_sdf.columns.to_list()
    # Columns to remove
    removal_list = ['OBJECTID', 'Shape__Area', 'Shape__Length', 'CreationDate', 'Creator', 'EditDate', 'Editor']
    current_columns = list(filter(lambda i: i not in removal_list, current_columns))
    sdf = sdf[current_columns]

    # Truncate
    layer.manager.truncate()
    
    # Convert DataFrame to FeatureSet
    addResults = []
    if sdf is not None and len(sdf)>0:
        print(f"Add Features {len(sdf)}")
        numAddLoops = math.floor(len(sdf)/chunkSize) + 1
        startAddNum = 0
        while numAddLoops > 0:
            minAddNum = startAddNum
            maxAddNum = startAddNum + chunkSize
            startAddNum = startAddNum + chunkSize
            numAddLoops = numAddLoops-1
            temp_fs = sdf[minAddNum:maxAddNum].copy()
            adds_featureset = FeatureSet.from_dataframe(temp_fs)
            addFeatures = layer.edit_features(adds=adds_featureset)
            addResults.append(addFeatures)
    else:
        print("No Records to Add")
    return addResults


def createFeatureService(gis, sdf, name, time_period, folder="Weather"):
    fs_name = f"{name} - {time_period}"
    results = sdf.spatial.to_featurelayer(title=fs_name, gis=gis, folder="Weather")
    return results


def createFeatureServiceDict(gis, name, time_period_dict):
    output_dict = {}
    time_period_list = list(time_period_dict.keys())
    for t in time_period_list:
        period_sdf = time_period_dict[t]
        new_service = createFeatureService(gis, period_sdf, name, t)
        item_id = str(new_service.id)
        output_dict[t] = item_id
        del item_id, new_service, period_sdf
    return output_dict



def compareLayer2SDF(layer, sdf, id_field = "GRID_ID", value_field="Value"):
    existing_sdf = pd.DataFrame.spatial.from_layer(layer)
    print(f"Existing SDF Number of Rows {existing_sdf.shape[0]}")
    print(f"New SDF Number of Rows {sdf.shape[0]}")
    existing_sdf = existing_sdf[[id_field, value_field]]
    print("Existing SDF")
    print(existing_sdf.head())
    print("Dups in Existing SDF")
    print(existing_sdf[existing_sdf.duplicated([id_field], keep=False)])
    print("Existing SDF After Set Index")
    existing_reindex = existing_sdf.set_index(id_field)
    #existing_sdf.set_index(id_field, inplace=True)
    print(existing_reindex.head())
    #existing_dict = existing_sdf.set_index(id_field).to_dict("index")
    existing_dict = existing_reindex.to_dict("index")
    print("Existing Dictonary")
    #print(existing_dict)
    # New Dictonary
    temp_sdf = sdf[[id_field, value_field]]
    new_dict = temp_sdf.set_index(id_field).to_dict("index")
    update_ids = []
    update_dict = {}
    for key in list(existing_dict.keys()):
        old_value = existing_dict[key][value_field]
        new_value = new_dict[key][value_field]
        if old_value != new_value:
            update_ids.append(key)
            update_dict[key] = new_value
    # Create Output SDF
    output_sdf = existing_sdf[existing_sdf[id_field].isin(update_ids)].copy()
    output_sdf[value_field] = output_sdf[id_field].apply(lambda x: update_dict[x])
    print(f"|-- Difference Detected {output_sdf.shape[0]} Rows")
    return output_sdf


def compareDataFrames(input_dict, result, time_period, gis):
    item_id = input_dict['Item IDs'][time_period]
    
    # Create a Spatially-Enabled DataFrame from the feature layer
    item = gis.content.get(item_id)
    layer = item.layers[0]
    sdf = pd.DataFrame.spatial.from_layer(layer)
    sdf.rename(columns={'Value':'Old Value'}, inplace=True)
    
    # Get the updated df
    df = result[time_period]
    
    # Merge
    updated_sdf = sdf.merge(df, how='left', left_on='grid_id', right_on = 'GRID_ID')
    
    # Calculate the Difference
    updated_sdf['Difference in Values'] = updated_sdf.apply(lambda row: row['New Value'] - row['Old Value'], axis=1)
    update_sdf = updated_sdf[updated_sdf['Difference in Values']!=0].copy()
    update_sdf = update_sdf[['grid_id', 'GlobalID', 'OBJECTID', 'New Value']]
    update_sdf.rename(columns={'New Value':'Value'}, inplace=True)
    return update_sdf


def updateFeatures(update_sdf, input_dict, result, time_period, gis, chunkSize=100):
    """
    Update Features in a specific ArcGIS Online Feature Service
    """
    item_id = input_dict['Item IDs'][time_period]
    
    # Create a Spatially-Enabled DataFrame from the feature layer
    item = gis.content.get(item_id)
    layer = item.layers[0]
    
    updateResults = []
    
    if update_sdf is not None and len(update_sdf)>0:
        print(f"Updating {len(update_sdf)} Features")
        numLoops = math.floor(len(update_sdf)/chunkSize)+1
        startAddNum = 0
        while numLoops > 0:
            minAddNum = startAddNum
            maxAddNum = startAddNum + chunkSize
            startAddNum = startAddNum + chunkSize
            numLoops = numLoops-1
            temp_sdf = update_sdf[minAddNum:maxAddNum].copy()
            update_fs = FeatureSet.from_dataframe(temp_sdf)
            # Apply Update Edits to Service
            try:
                update_results = layer.edit_features(updates=update_fs)
            except Exception as e:
                print(f"ERROR TRYING TO UPDATE FEATURE SERVICE - {str(e)}")
                print(f"Will Try Again in 5 Seconds")
                time.sleep(5.0)
                try:
                    update_results = layer.edit_features(updates=update_fs)
                except Exception as e:
                    print(f"FAILED TO UPDATE A SECOND TIME")
                    print(f"{str(e)}")
                    update_results = []                                              
            updateResults.append(update_results)
            del temp_sdf, update_fs
    else:
        print("No Records to Update")
    return updateResults



def processGRIB(input_dict, input_hexbin, hex_sdf, all_periods = "All Time Periods", keep_intermediate_dfs=False):
    """
    Process NDFD GRIB Files and Summarizes by HexBin
    Parameters:
        input_dict (dict): A dictonary that describes the input NDFD data, intermediate ouptuts, and feature services to be updated
        input_hexbin (str): The path and name of the hexbin layer
        hex_sdf (pd.DataFrame.spatial): A spatially enabable DataFrame that represents the hexbins
        all_periods (str): The time category that is used to summarize all the relevant time periods
        keep_intermeidate_dfs (bool): Flag to indiciate if the intermediate dataframes should be returned by this function
    Returns:
        Dictonary: A dictonary with one or two elements if keep_intermeidate_dfs. This dictonary includes the output DataFrames
    
    """
    if keep_intermediate_dfs:
        intermediate_dict = {}
    updates_dict = {}
    name = input_dict['Name']
    input_url = input_dict['URL']
    output_layer = input_dict['Output Layer']
    output_table = input_dict['Output Table']
    #output_table = os.path.join(arcpy.env.scratchGDB, output_table_name)
    print(f"Output Table Check: {output_table}")
    
    variable = input_dict['Variable']
    summary_type = input_dict['Summary Type']
    item_ids = input_dict['Item IDs']
    if 'Conversion' in list(input_dict.keys()):
        conversion_type = input_dict['Conversion']
    else:
        conversion_type = None
    if 'Rounding' in list(input_dict.keys()):
        rounding = int(input_dict['Rounding'])
    else:
        rounding = None
    grib_file = downloadGRIB(input_url, download_folder)
    raster_layer = makeRasterLayer(grib_file, output_layer, variable)
    summary_table = runZonalStats(input_hexbin, raster_layer, output_table, id_field="GRID_ID")
    arcpy.management.Delete(raster_layer)
    sdf = pd.DataFrame.spatial.from_table(summary_table)
    sdf['Local Time'] = sdf['StdTime'].dt.tz_localize("UTC").dt.tz_convert('America/Los_Angeles').dt.tz_localize(None)
    #sdf['Local Time'] = sdf['StdTime']
    sdf['Time Period'] = sdf['Local Time'].apply(lambda t: categorizeStdTime(t))
    print(f"Unique Time Periods")
    for tp in list(sdf['Time Period'].unique()):
        print(f"| -- {tp}")
    # Apply Conversions    
    if conversion_type is not None:
        if conversion_type == 'Kelvin to Fahrenheit':
            sdf[summary_type] = sdf[summary_type].apply(lambda x: kelvin2Fahrenheit(x))
        elif conversion_type == 'Meters per Second to Miles per Hour':
            sdf[summary_type] = sdf[summary_type].apply(lambda x: meters2mph(x))
        else:
            print("ERROR - Unknown Conversion Type Specified, Please Try Again")
    if rounding is not None:
        sdf[summary_type] = sdf[summary_type].apply(lambda x: round(float(x), rounding))
    if keep_intermediate_dfs:
        intermediate_dict[name] = sdf
    sdf_dict = summarizeTimePeriods(sdf, summary_type, all_periods)
    # Check to See if the Intermeddiate DataFrams should be included?
    if keep_intermediate_dfs:
        sdf_dict = {'Summary':sdf_dict, 'Intermediate':intermediate_dict}
    return sdf_dict

def createPlot(variable, intermediate_dict, results_dict, x, y, xlabel, ylabel, title, hexbin_id='FO-122', all_periods='Entire 3 Day Period', annotation_adj = 0.5):
    """
    Create plot to visualize the NDFD Summary by hour for a single hexbin.
    """
    # print(f"Creating Plot for {variable}")
    # Intermediate/Source DataFrame
    intermediate_df = intermediate_dict[variable][variable]
    intermediate_df = intermediate_df[intermediate_df['grid_id'] == hexbin_id]
    
    # Set up Plot
    sns.set_style("darkgrid")
    #plt.style.use("dark_background")
    plt.figure(figsize=(24,6))
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    
    # Create Intial Line Plot
    temp = sns.lineplot(data=intermediate_df, x=x, y=y)
    temp.xaxis.set_major_locator(mdates.HourLocator(interval=1))
    # set formatter
    temp.xaxis.set_major_formatter(mdates.DateFormatter('%H'))
    
    # Lets go through each of the time periods in the results
    time_periods = list(results_dict[variable].keys())
    for t in time_periods:
        if t != all_periods:
            if t == 'Today':
                minDate = datetime.now(pytz.timezone('America/Los_Angeles')).date()
                maxDate = datetime.now(pytz.timezone('America/Los_Angeles')).date() + timedelta(days = 1)
                temp.axvspan(xmin=minDate, xmax=maxDate, facecolor='white', alpha=0.5)
                # Get Max for Today
                temp_df = results_dict[variable][t]
                if temp_df.shape[0] > 0:
                    value = temp_df.loc[temp_df['GRID_ID'] == hexbin_id, 'New Value'].values[0]
                    # Text Annotation
                    xvalue = intermediate_df.loc[(intermediate_df['grid_id'] == hexbin_id) & (intermediate_df[y] == value) & (intermediate_df[x].dt.date == minDate), x].values
                    try:
                        temp.text(xvalue[0], value+annotation_adj, f'{minDate.strftime("%m/%d/%Y")} {y.title()}: {str(value)}')
                    except Exception as e:
                        print(f"Error Adding Annotation {str(e)}")
                        print(f"Potential Date/Times - {xvalue}")
                        print(f"Max/Min Value - {value}")
                        print(f"Checking Filters on Source DataFame")
                        print(f"| --- Columns on intermediate df {intermediate_df.columns.to_list()}")
                        print(f"| --- Number of Rows with Value Filter {intermediate_df[intermediate_df[y] == value].shape[0]}")
                        print(f"| --- Number of Rows with grid_id Filter {intermediate_df[intermediate_df['grid_id'] == hexbin_id].shape[0]}")
                        print(f"| --- | --- Rows with Value and Grid ID Filters {intermediate_df[(intermediate_df['grid_id'] == hexbin_id) & (intermediate_df[y] == value)].shape[0]}")
                        print(f"| --- Number of Rows with Date Filter {intermediate_df[intermediate_df[x].dt.date == minDate].shape[0]}")
                        print(f"| --- | --- Rows with Value, Grid ID, and Date Filters {intermediate_df[(intermediate_df['grid_id'] == hexbin_id) & (intermediate_df[y] == value) & (intermediate_df[x].dt.date == minDate)].shape[0]}")
                        print(f"Looking at the potential dates")
                        print(f"|-- {intermediate_df.loc[intermediate_df[y] == value, x].values}")
                        print(f"|-- Min Date {minDate.strftime('%m/%d/%Y')}")
            elif t == '2 Days':
                minDate = datetime.now(pytz.timezone('America/Los_Angeles')).date() + timedelta(days = 2)
                maxDate = datetime.now(pytz.timezone('America/Los_Angeles')).date() + timedelta(days = 3)
                temp.axvspan(xmin=minDate, xmax=maxDate, facecolor='white', alpha=0.5)
                # Get Max for Today
                temp_df = results_dict[variable][t]
                if temp_df.shape[0] > 0:
                    value = temp_df.loc[temp_df['GRID_ID'] == hexbin_id, 'New Value'].values[0]
                    # Text Annotation
                    # Text Annotation
                    xvalue = intermediate_df.loc[(intermediate_df['grid_id'] == hexbin_id) & (intermediate_df[y] == value) & (intermediate_df[x].dt.date == minDate), x].values
                    try:
                        #xvalue = intermediate_df.loc[(intermediate_df['grid_id'] == hexbin_id) & (intermediate_df['MAX'] == value), 'Local Time'].values[0]
                        temp.text(xvalue[0], value+annotation_adj, f'{minDate.strftime("%m/%d/%Y")} {y.title()}: {str(value)}')
                    except Exception as e:
                        print(f"Error Adding Annotation {str(e)}")
                        print(f"Potential Date/Times - {xvalue}")
                        print(f"Max/Min Value - {value}")
                        print(f"Checking Filters on Source DataFame")
                        print(f"| --- Columns on intermediate df {intermediate_df.columns.to_list()}")
                        print(f"| --- Number of Rows with Value Filter {intermediate_df[intermediate_df[y] == value].shape[0]}")
                        print(f"| --- Number of Rows with grid_id Filter {intermediate_df[intermediate_df['grid_id'] == hexbin_id].shape[0]}")
                        print(f"| --- | --- Rows with Value and Grid ID Filters {intermediate_df[(intermediate_df['grid_id'] == hexbin_id) & (intermediate_df[y] == value)].shape[0]}")
                        print(f"| --- Number of Rows with Date Filter {intermediate_df[intermediate_df[x].dt.date == minDate].shape[0]}")
                        print(f"| --- | --- Rows with Value, Grid ID, and Date Filters {intermediate_df[(intermediate_df['grid_id'] == hexbin_id) & (intermediate_df[y] == value) & (intermediate_df[x].dt.date == minDate)].shape[0]}")
                        print(f"Looking at the potential dates")
                        print(f"|-- {intermediate_df.loc[intermediate_df[y] == value, x].values}")
                        print(f"|-- Min Date {minDate.strftime('%m/%d/%Y')}")
            else:
                res = [int(i) for i in t.split() if i.isdigit()]
                if len(res) > 0:
                    day_adjust = res[0]
                    minDate = datetime.now(pytz.timezone('America/Los_Angeles')).date() + timedelta(days = day_adjust)
                    # Get Max for Today
                    temp_df = results_dict[variable][t]
                    if temp_df.shape[0] > 0:
                        value = temp_df.loc[temp_df['GRID_ID'] == hexbin_id, 'New Value'].values[0]
                        # Add Text
                        xvalue = intermediate_df.loc[(intermediate_df['grid_id'] == hexbin_id) & (intermediate_df[y] == value) & (intermediate_df[x].dt.date == minDate), x].values
                        try:
                            temp.text(xvalue[0], value+annotation_adj, f'{minDate.strftime("%m/%d/%Y")} {y.title()}: {str(value)}')
                        except Exception as e:
                            print(f"Error Adding Annotation {str(e)}")
                            print(f"Potential Date/Times - {xvalue}")
                            print(f"Max/Min Value - {value}")
                            print(f"Checking Filters on Source DataFame")
                            print(f"| --- Columns on intermediate df {intermediate_df.columns.to_list()}")
                            print(f"| --- Number of Rows with Value Filter {intermediate_df[intermediate_df[y] == value].shape[0]}")
                            print(f"| --- Number of Rows with grid_id Filter {intermediate_df[intermediate_df['grid_id'] == hexbin_id].shape[0]}")
                            print(f"| --- | --- Rows with Value and Grid ID Filters {intermediate_df[(intermediate_df['grid_id'] == hexbin_id) & (intermediate_df[y] == value)].shape[0]}")
                            print(f"| --- Number of Rows with Date Filter {intermediate_df[intermediate_df[x].dt.date == minDate].shape[0]}")
                            print(f"| --- | --- Rows with Value, Grid ID, and Date Filters {intermediate_df[(intermediate_df['grid_id'] == hexbin_id) & (intermediate_df[y] == value) & (intermediate_df[x].dt.date == minDate)].shape[0]}")
                            print(f"Looking at the potential dates")
                            print(f"|-- {intermediate_df.loc[intermediate_df[y] == value, x].values}")
                            print(f"|-- Min Date {minDate.strftime('%m/%d/%Y')}")
        else:
            temp_df = results_dict[variable][t]
            value = temp_df.loc[temp_df['GRID_ID'] == hexbin_id, 'New Value'].values[0]
            # Add line to indicate max over time period
            temp.axhline(value, ls='--', c='red')
            temp.text(datetime.now(pytz.timezone('America/Los_Angeles')).date(), value+annotation_adj, f"{y.title()} Value Over 3 Day Period: {str(value)}", c='red')

In [ ]:
# Dictonary to Store Results
results_dict = {}
# Dictonary to Store Intermediate DataFrames
intermediate_dict = {}

# Process GRIBs for Each Input
for input_dict in inputs_dicts:
    input_name = input_dict['Name']
    print(f"PROCESSING - {input_name}")
    result = processGRIB(input_dict, input_hexbin, hex_sdf, "Entire 3 Day Period", keep_intermediate_dfs=True)
    # Add Results to Results Dictonary
    results_dict[input_name] = result['Summary']
    # Add Intermediate DFs to Intermediate Dictonary
    intermediate_dict[input_name] = result['Intermediate']

### Create Some Plots for QA/QC - Forecast Time Series Plots for Downtown Sacramento (FO-122)

In [ ]:
createPlot('Maximum Daily Temperature (NDFD)', intermediate_dict, results_dict, x='Local Time', y='MAX', xlabel="Time of Day (Hours)", ylabel="Maximum Hourly Temperature (Fahrenheit)", title="NDFD Forecasted Maximum Temperature by Time of Day", hexbin_id='FO-122', all_periods='Entire 3 Day Period', annotation_adj = 0.5)

In [ ]:
createPlot('Maximum Daily Wind Speed (NDFD)', intermediate_dict, results_dict, x='Local Time', y='MAX', xlabel="Time of Day (Hours)", ylabel="Maximum Hourly Wind Speed (Miles Per Hour)", title="NDFD Forecasted Maximum Wind Speed by Time of Day", hexbin_id='FO-122', all_periods='Entire 3 Day Period', annotation_adj = 0.1)

In [ ]:
createPlot('Minimum Daily Relative Humidity (NDFD)', intermediate_dict, results_dict, x='Local Time', y='MIN', xlabel="Time of Day (Hours)", ylabel="Minimum Hourly Relative Humidity (%)", title="NDFD Forecasted Minimum Relative Humidity by Time of Day", hexbin_id='FO-122', all_periods='Entire 3 Day Period', annotation_adj = -2)

In [ ]:
# List to Store the Update Results
updateResults = []

# Process Each NDFD Element
for input_dict in inputs_dicts:
    input_name = input_dict['Name']
    print(f"Processing {input_name}")
    result = results_dict[input_name]
    print(f"Input has {len(result)} DataFrames")
    for time_period in list(input_dict['Item IDs'].keys()):
        item_id = input_dict['Item IDs'][time_period]
        update_sdf = compareDataFrames(input_dict, result, time_period, gis)
        print(f"|-- Time Period {time_period} has {len(update_sdf)} rows to update")
        update_results = updateFeatures(update_sdf, input_dict, result, time_period, gis)
        del update_sdf, item_id
        updateResults.append(update_results)
    del result, input_name
        
        
                

In [ ]:
# Delete files in downlaod folder
for f in os.listdir(download_folder):
    delete_file = os.path.join(download_folder, f)
    print(f"Removing - {delete_file}")
    try:
        os.remove(delete_file)
    except Exception as e:
        print(f"Error Removing NDFD - {str(e)}")

# Delete FGDB
try:
    arcpy.Delete_management("/arcgis/home/weather_data/ZonalTables.gdb")
    print("Removed ZonalTables FGDB")
except Exception as e:
    print(f"Error Deleting FGDB - {str(e)}")
    

In [ ]:
print("Finished!")
end_time = datetime.now()
print(f"Process Complete - Took {round((end_time-start_time).seconds/60,2)} Minutes to Run")